<a href="https://colab.research.google.com/github/Infor-Mayo/colab_flux/blob/main/flux.1-dev_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 14652, done.
remote: Counting objects: 100% (3401/3401), done.
remote: Compressing objects: 100% (307/307), done.
remote: Total 14652 (delta 3244), reused 3094 (delta 3094), pack-reused 11251 (from 1)
Receiving objects: 100% (14652/14652), 22.10 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (9939/9939), done.
/content/TotoroUI
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.1 MB/s eta 0:00:00
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123623 files and directories currentl

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
!pip install pyngrok

ingresa alado de !ngrok authtoken el id de ngrok, deves crearte una cuenta es gratis

In [4]:
!echo -n > /root/.config/ngrok/ngrok.yml
!cat /root/.config/ngrok/ngrok.yml
!ngrok authtoken "2oDUgJpSBQDvZ7XRqH8k2WGjyof_52TNeRcKFmQ3q2YD64hMM"
!cat /root/.config/ngrok/ngrok.yml



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
version: "3"
agent:
    authtoken: 2oDUgJpSBQDvZ7XRqH8k2WGjyof_52TNeRcKFmQ3q2YD64hMM


In [5]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(5001)  # Asegúrate de que el puerto sea 5000 o el que estés usando para Flask
print("URL pública:", public_url)



URL pública: NgrokTunnel: "https://5298-35-230-43-197.ngrok-free.app" -> "http://localhost:5001"


Arriba esta la url publica que deves ingresar despues de ejecutar los dos de abajo !!!


In [7]:
titulo = ""
descripcion = ""
alto = ""
ancho = ""
etapas = ""

In [8]:
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)


@app.route('/texto', methods=['POST'])
def recibir_texto():
    global titulo, descripcion,alto,ancho,etapas
    data = request.json
    titulo = data.get('titulo')
    descripcion = data.get('descripcion')
    alto = data.get('alto')
    ancho = data.get('ancho')
    etapas = data.get('etapas')
    print("Título recibido:", titulo)  # Imprime el título
    print("Descripción recibida:", descripcion)  # Imprime la descripción
    return jsonify({"status": "texto recibido", "titulo": titulo, "descripcion": descripcion})

def iniciar_servidor():
    app.run(port=5001)  # Cambia a 5001

# Iniciar el servidor en un hilo separado
threading.Thread(target=iniciar_servidor).start()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit


Corre hasta akí ...

In [ ]:
print(titulo,descripcion)

crearcuentaviajando ## Miniatura del Video: Creando una Cuenta en Viajando para Comprar Pasajes en Cuba

**Descripción visual:**

* **Personajes:** El video se centra en el usuario que está creando una cuenta en la aplicación "Viajando" y muestra un fragmento de la pantalla donde se ven las opciones de registro. Se puede mostrar una imagen con una mano seleccionando un botón o un icono del app "Viajando". 
* **Escena:** La miniatura debe mostrar una pantalla de la aplicación, con el logo de la app (un asiento) en el centro y la interfaz de la cuenta en la parte inferior.  Podría incluir una sección con información sobre términos y condiciones.
* **Acciones:** Se puede mostrar un "botón" que se está presionando con la mano en la pantalla, y luego una imagen que muestre un correo electrónico con un enlace de confirmación.
* **Emoción:** La miniatura debe transmitir sensación de progreso y satisfacción al usuario. 

**Título llamativo:** 🇨🇺 ¡Es Fácil Comprar Pasajes en Cuba!

**Estilo de 

Genera la imagen y la descarga en tu pc

In [ ]:
from google.colab import files
import random
import numpy as np
from PIL import Image

with torch.inference_mode():
    positive_prompt = descripcion
    width = ancho
    height = alto
    seed = 0
    steps = etapas
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"Seed utilizado: {seed}")

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    # Guardar la imagen en el directorio de Colab
    image_path = f"/content/{titulo}.png"
    Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0]).save(image_path)

# Descargar la imagen generada
files.download(image_path)


Seed utilizado: 4036071985724541206


  0%|          | 0/20 [00:00<?, ?it/s]